# 0. Load required modules.

In [20]:
!pip install numpy==1.21.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 33.9 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.4
    Uninstalling numpy-1.18.4:
      Successfully uninstalled numpy-1.18.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jiant 2.2.0 requires attrs==19.3.0, but you have attrs 23.2.0 which is incompatible.
jiant 2.2.0 requires numpy==1.18.4, but you have numpy 1.21.6 which is incompatible.
torchvision 0.9.1 requires torch==1.8.1, but you have torch 2.0.1 which is incompatible.


In [21]:
import datasets
import pickle
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig, RobertaForSequenceClassification, RobertaForMaskedLM
import torch
import numpy as np
from tqdm import tqdm_notebook

In [70]:
path = './validation/wsc/'

# 1. Load the generated dataset.

In [71]:
with open(path+'/data_pos.pickle', 'rb') as f:
    data_pos = pickle.load(f)
with open(path+'/data_set1.pickle', 'rb') as f:
    data_set1 = pickle.load(f)    
with open(path+'/data_set2.pickle', 'rb') as f:
    data_set2 = pickle.load(f)

# 2. Load the trained model.

In [75]:
model_file = '/home/skhong/jiant/roberta/wsc/runs/simple/best_model.p'
config_file = '/home/skhong/jiant/roberta/cb/models/roberta-base/model/config.json'

model_version = 'roberta-base'
config = RobertaConfig.from_json_file(config_file)
model = RobertaForSequenceClassification(config).cuda()

state_dict = torch.load(model_file)
new_dict = dict()

lst = ['encoder.embeddings.position_ids', 'encoder.embeddings.word_embeddings.weight', 'encoder.embeddings.position_embeddings.weight', 'encoder.embeddings.token_type_embeddings.weight', 'encoder.embeddings.LayerNorm.weight', 'encoder.embeddings.LayerNorm.bias']
lst2 = ["taskmodels_dict.wic.head.span_attention_extractor._global_attention._module.weight", "taskmodels_dict.wic.head.span_attention_extractor._global_attention._module.bias"]
lst3 = ["taskmodels_dict.wic.head.classifier.weight", "taskmodels_dict.wic.head.classifier.bias"]
for key in state_dict.keys():
    if 'taskmodels_dict.wsc.encoder.encoder' in key:
        new_dict['roberta.'+key[28:]] = state_dict[key]
    elif 'encoder.encoder' in key:
        new_dict['roberta.'+key[8:]] = state_dict[key]
    elif 'taskmodels_dict.wsc.head.' in key:
        new_dict['classifier.'+key[25:]] = state_dict[key]
    elif key in lst:
        new_dict['roberta.'+key[8:]] = state_dict[key]
model.load_state_dict(new_dict,strict=False)
tokenizer = RobertaTokenizer.from_pretrained(model_version)

In [9]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50265, 768, padding_idx=1)

# 3. Extract attention values between tokens.
- For models with WI added, perform the addition and extract attention values; for cases where it's not the case, input the excluded ones and extract attention values.

In [76]:
X = []
Y = []

model.eval()

for i, data in tqdm_notebook(enumerate(data_pos), total=len(data_pos)):
    input_ids = data[0]
    token_type_ids = data[1]
    tfidf_ids = data[2]
    
    i_pos = data_set1[i][0]
    j_pos = data_set1[i][1]
    j_random_pos = data_set2[i][1]
    
    with torch.no_grad():
        pred = model(input_ids, attention_mask=token_type_ids)
        
    temp_x1 = []
    temp_x2 = []
    
    # 128 is the maximum input length for the model's text.
    for k in range(12):
        temp_data = pred.attentions[k].detach().cpu().numpy().reshape(12, 512, 512)
        try:
            for l in range(12):
                temp_x1.append(temp_data[l][i_pos][j_pos])
                temp_x2.append(temp_data[l][i_pos][j_random_pos])
        except:
            continue
    
    Y.append(1)
    X.append(temp_x1)
    Y.append(0)
    X.append(temp_x2)
    
X = np.array(X)
Y = np.array(Y)

/tmp/ipykernel_37127/941288186.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, data in tqdm_notebook(enumerate(data_pos), total=len(data_pos)):


# 4. WI Validation Model based Performance Evaluation

In [83]:
from sklearn.model_selection import train_test_split
print(len(Y))
print(len(X))
#Y = np.array(map(int,Y))
new_X = list()
new_Y = list()
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, stratify=Y, random_state=34)

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
clf = RandomForestClassifier(
            n_estimators=50, 
            criterion='entropy', 
            max_depth=5, 
            max_features='sqrt',
            max_samples=0.1,
            bootstrap=True,
            oob_score=True, 
            random_state=100
        ).fit(x_train,y_train)

## Performance Evaluation
print('ACC : ', clf.score(x_test,y_test))

360
360
ACC :  0.8055555555555556
